### Задание № 1

In [1]:
pip install gensim

  Using cached smart_open-5.2.1-py3-none-any.whl (58 kB)
Note: you may need to restart the kernel to use updated packages.


In [110]:
import gensim
import numpy as np
import pandas as pd
from datetime import datetime
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
from sklearn.decomposition import TruncatedSVD, NMF
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedKFold
from string import punctuation
from razdel import tokenize as razdel_tokenize
import pickle
import warnings
morph = MorphAnalyzer()
warnings.filterwarnings("ignore")

In [62]:
def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word]
    normalized_text = [morph.parse(word)[0].normal_form for word in normalized_text]
    return ' '.join(normalized_text)

In [63]:
data = pd.read_csv('avito_category_classification.csv')
data['description_norm'] = data['description'].apply(normalize)

In [64]:
vectorizer = TfidfVectorizer(max_features=2000, min_df=10, max_df=0.4)
X = vectorizer.fit_transform(data['description_norm'])

In [81]:
pipelines = {
    'svd_sgd' : Pipeline([
    ('bow', vectorizer),
    ('svd', TruncatedSVD(500)),
    ('clf', SGDClassifier(max_iter=1000, tol=1e-3))]),
        
    'svd_kn' : Pipeline([
    ('bow', vectorizer),
    ('svd', TruncatedSVD(500)),
    ('clf', KNeighborsClassifier(n_neighbors=7))]),
        
    'svd_rf' : Pipeline([
    ('bow', vectorizer),
    ('svd', TruncatedSVD(500)),
    ('clf', RandomForestClassifier(n_estimators=100, max_depth=10))]),
        
    'svd_extratrees' : Pipeline([
    ('bow', vectorizer),
    ('svd', TruncatedSVD(500)),
    ('clf', ExtraTreesClassifier(random_state=0))]),
   
    'nmf_sgd' : Pipeline([
    ('bow', vectorizer),
    ('svd', NMF(60)),
    ('clf', SGDClassifier(max_iter=1000, tol=1e-3))]),

    'nmf_kn' : Pipeline([
    ('bow', vectorizer),
    ('svd', NMF(60)),
    ('clf', KNeighborsClassifier(n_neighbors=7))]),
        
    'nmf_rf' : Pipeline([
    ('bow', vectorizer),
    ('svd', NMF(60)),
    ('clf', RandomForestClassifier(n_estimators=100, max_depth=10))]),

    'nmf_extratrees' : Pipeline([
    ('bow', vectorizer),
    ('svd', NMF(60)),
    ('clf', ExtraTreesClassifier(random_state=0))]),
}

In [82]:
def eval_table(X, y, pipeline, N=6):
    labels = list(set(y))
 
    fold_metrics = pd.DataFrame(index=labels)
    errors = np.zeros((len(labels), len(labels)))

    kfold = StratifiedKFold(n_splits=N, shuffle=True, )
    
    for i, (train_index, test_index) in enumerate(kfold.split(X, y)):
        pipeline.fit(X[train_index], y[train_index])
        preds = pipeline.predict(X[test_index])

        fold_metrics[f'precision_{i}'] = precision_score(y[test_index], preds, labels=labels, average=None)
        fold_metrics[f'recall_{i}'] = recall_score(y[test_index], preds, labels=labels, average=None)
        fold_metrics[f'f1_{i}'] = f1_score(y[test_index], preds, labels=labels, average=None)
        errors += confusion_matrix(y[test_index], preds, labels=labels, normalize='true')

    result = pd.DataFrame(index=labels)
    result['precision'] = fold_metrics[[f'precision_{i}' for i in range(N)]].mean(axis=1).round(2)
    result['precision_std'] = fold_metrics[[f'precision_{i}' for i in range(N)]].std(axis=1).round(2)
    
    result['recall'] = fold_metrics[[f'recall_{i}' for i in range(N)]].mean(axis=1).round(2)
    result['recall_std'] = fold_metrics[[f'recall_{i}' for i in range(N)]].std(axis=1).round(2)
    
    result['f1'] = fold_metrics[[f'f1_{i}' for i in range(N)]].mean(axis=1).round(2)
    result['f1_std'] = fold_metrics[[f'f1_{i}' for i in range(N)]].std(axis=1).round(2)
 
    result.loc['mean'] = result.mean().round(2)
    errors /= N
    
    return result, errors

In [101]:
for count, pipe in enumerate(pipelines):
    metrics_svd, errors_svd = eval_table(data['description_norm'], data['category_name'], pipelines[pipe])
    print(f'{pipe}: {metrics_svd.loc["mean"]["f1"].round(3)}\n', end='')

svd_sgd: 0.76
svd_kn: 0.48
svd_rf: 0.56
svd_extratrees: 0.62
nmf_sgd: 0.51
nmf_kn: 0.52
nmf_rf: 0.61
nmf_extratrees: 0.69


**лучшее сочетание - svd_sgd**

### Задание № 2 

In [107]:
texts = open('wiki_data.txt', encoding='utf-8').read().splitlines()[:5000]

In [108]:
texts = ([normalize(text) for text in texts])

In [111]:
text = [text.split() for text in texts]
ph = gensim.models.Phrases(texts, scoring='npmi', threshold=0.4) # threshold можно подбирать
p = gensim.models.phrases.Phraser(ph)
ngrammed_texts = p[texts]

In [118]:
dictinary = gensim.corpora.Dictionary((text.split() for text in texts))
dictinary.filter_extremes(no_above=0.1, no_below=10)
dictinary.compactify()

print(dictinary)

Dictionary(8102 unique tokens: ['1,2', '1,5', '12', '14', '16']...)


In [113]:
corpus = [dictinary.doc2bow(text.split()) for text in texts]
lda = gensim.models.LdaMulticore(corpus, 
                                 100, # колиество тем
                                 alpha='asymmetric',
                                 id2word=dictinary, 
                                 passes=10)
lda.print_topics()

[(99,
  '0.006*"случай" + 0.005*"звезда" + 0.005*"вода" + 0.005*"использовать" + 0.005*"друг" + 0.005*"температура" + 0.004*"собака" + 0.004*"форма" + 0.004*"различный" + 0.004*"смотреть"'),
 (98,
  '0.020*"монастырь" + 0.007*"маргарита" + 0.006*"пещера" + 0.006*"книга" + 0.006*"колония" + 0.005*"действие" + 0.005*"монастырский" + 0.005*"день" + 0.005*"персонаж" + 0.004*"дом"'),
 (97,
  '0.024*"участок" + 0.021*"строительство" + 0.015*"земельный" + 0.012*"план" + 0.011*"объект" + 0.010*"проект" + 0.010*"сооружение" + 0.009*"рубль" + 0.009*"опора" + 0.008*"мост"'),
 (96,
  '0.017*"свидетель" + 0.012*"германия" + 0.010*"гитлер" + 0.008*"движение" + 0.007*"немецкий" + 0.007*"полигон" + 0.007*"военный" + 0.007*"метр" + 0.006*"дистанция" + 0.005*"сила"'),
 (95,
  '0.023*"картина" + 0.016*"рим" + 0.015*"мария" + 0.014*"римский" + 0.013*"медичи" + 0.008*"изобразить" + 0.007*"италия" + 0.007*"портрет" + 0.006*"папа" + 0.006*"художник"'),
 (94,
  '0.044*"армия" + 0.026*"войско" + 0.021*"флот" +

In [114]:
np.exp2(-lda.log_perplexity(corpus[:1000]))

5963.936381005242

In [115]:
topics = []
for topic_id, topic in lda.show_topics(num_topics=100, formatted=False):
    topic = [word for word, _ in topic]
    topics.append(topic)
coherence_model_lda = gensim.models.CoherenceModel(topics=topics, 
                                                   texts=[text.split() for text in texts], 
                                                   dictionary=dictinary, coherence='c_v')
coherence_model_lda.get_coherence()

0.5416919427765893

In [117]:
[text for text in ngrammed_texts[:3]]

['новостройка нижегородский область новостро́йка — сельский посёлок в дивеевский район нижегородский область входить в состав сатисский сельсовет посёлок расположить в 12,5 км к юг от село дивеево и 1 км к запад от город саров на право берег река вичкинза правый приток река сатис окружить смешанный лес соединить асфальтовый дорога с посёлок цыгановка 1,5 км и грунтовый просёлочный дорога с посёлок сатис 3,5 км название новостройка являться сугубо официальный местный население использовать исключительно альтернативный название — хитрый употребляться языковой оборот « … на хитрый » ранее использовать название — песчаный известковый основать в 1920-й год переселенец из соседний село аламасовый и нарышкино расположить соответственно в 8 и 14 км к запад в вознесенский район традиционно в посёлок жить рабочий совхоз « вперёд » центр в посёлок сатис возле посёлок расположить карьер где активно добывать доломитовый мука и бутовый камень в настоящий время официально закрытый по данные 1978 год 

In [136]:
ngrammed_dictinary = gensim.corpora.Dictionary((text.split() for text in ngrammed_texts))
ngrammed_dictinary.filter_extremes(no_above=0.05, no_below=10)
ngrammed_dictinary.compactify()

In [ ]:
ngrammed_corpus = [dictinary.doc2bow(text.split()) for text in ngrammed_texts]
ngrammed_lda = gensim.models.LdaMulticore(corpus, 
                                 100, # колиество тем
                                 alpha='asymmetric',
                                 id2word=ngrammed_dictinary, 
                                 passes=5) 
ngrammed_lda.print_topics()

In [125]:
np.exp2(-lda.log_perplexity(corpus[:1000]))

5961.531966245532

In [ ]:
coherence_model_ngrammed_lda = gensim.models.CoherenceModel(topics=ngrammed_topics, 
                                                   texts=[text.split() for text in ngrammed_texts], 
                                                   dictionary=ngrammed_dictinary, coherence='c_v')
coherence_model_ngrammed_lda.get_coherence()

In [ ]:
dictinary = gensim.corpora.Dictionary((text.split() for text in texts))
dictinary.filter_extremes(no_above=0.1, no_below=10)
dictinary.compactify()

print(dictinary)

In [ ]:
tfidf = gensim.models.TfidfModel(corpus, id2word=dictinary, )
corpus_tfidf = tfidf[corpus]

lda_tfidf = gensim.models.LdaMulticore(corpus_tfidf, 
                                 100, # колиество тем
                                 alpha='asymmetric',
                                 id2word=dictinary, 
                                 passes=10) 
lda_tfidf.print_topics()